In [1]:
#  dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# reading in beer ratings csv

df=pd.read_csv(r'data\beer_rating.csv')
df.head()


,beer name,id,beer type,Name,brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


In [61]:
# creating brewery df based on the beer ratings csv to create a universaL index for all breweries
df = df[['brewery']].drop_duplicates()
ratings_brew_df = pd.DataFrame(df.groupby('brewery'))
ratings_brew_df.head()
grouped_rdf = pd.DataFrame()
grouped_rdf['brewery_name'] = ratings_brew_df[0]

grouped_rdf['brewery_name'] = grouped_rdf['brewery_name'].str.replace(')', '')
grouped_rdf['brewery_name'] = grouped_rdf['brewery_name'].str.replace('(', '')


grouped_rdf.reset_index()


<ipython-input-61-ff73ddc4c937>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  grouped_rdf['brewery_name'] = grouped_rdf['brewery_name'].str.replace(')', '')
<ipython-input-61-ff73ddc4c937>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  grouped_rdf['brewery_name'] = grouped_rdf['brewery_name'].str.replace('(', '')


,index,brewery_name
0,0,512 Brewing Company
1,1,10 Barrel Brewing Co.
2,2,14th Star Brewing Co.
3,3,18th Street Brewery - Gary Taproom
4,4,1940's Brewing Company
...,...,...
1493,1493,de Garde Brewing
1494,1494,pFriem Family Brewers
1495,1495,von Trapp Brewing
1496,1496,Ölvisholt Brugghús


In [77]:
# df based on open brewery db, to create a universal index for all breweries
bdf = pd.read_csv(r'data/breweries.csv')
bdf = bdf[['name']].drop_duplicates()
bdf = pd.DataFrame(bdf.groupby('name'))
grouped_bdf = pd.DataFrame()
grouped_bdf['brewery_name'] = bdf[0]

grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace('(', '')
grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace(')', '')

grouped_bdf.reset_index()

<ipython-input-77-f66129ea17aa>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace('(', '')
<ipython-input-77-f66129ea17aa>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace(')', '')


,index,brewery_name
0,0,405 Brewing Co
1,1,512 Brewing Co
2,2,10 Barrel Brewing Co
3,3,10 Barrel Brewing Co - Bend Pub
4,4,10 Barrel Brewing Co - Boise
...,...,...
7886,7886,Zwei Brewing Co
7887,7887,Zydeco Brew Werks
7888,7888,Zymurcracy Beer Company
7889,7889,Zymurgy Brew Works


In [83]:
# merge based on brewery name

merged_df = pd.merge(grouped_rdf, grouped_bdf, on=['brewery_name'],how='outer')
merged_df = merged_df.drop_duplicates('brewery_name').reset_index().rename(columns={'index':'brewery_id'})
merged_df.head()


,brewery_id,brewery_name
0,0,512 Brewing Company
1,1,10 Barrel Brewing Co.
2,2,14th Star Brewing Co.
3,3,18th Street Brewery - Gary Taproom
4,4,1940's Brewing Company


In [93]:
# create beer to brewery df
rdf = pd.read_csv(r'data/beer_rating.csv')
rdf = rdf[['id','beer name','brewery']]
rdf[['beer_id','beer_name','brewery_name']] = rdf[['id','beer name','brewery']]

beer_to_brewery_df = pd.merge(rdf,merged_df,on=['brewery_name'])
beer_to_brewery_df['brewery'] = merged_df['brewery_id']
beer_to_brewery_df = beer_to_brewery_df.drop(columns=['brewery'])
beer_to_brewery_df = beer_to_brewery_df[['brewery_name','brewery_id','beer_name','beer_id']]
beer_to_brewery_df.head()

id_df = beer_to_brewery_df[['brewery_id','beer_id']]
id_df.head()

,brewery_id,beer_id
0,34,251
1,34,71
2,34,442
3,34,1907
4,34,4316


In [ ]:
# create brewery info df

In [ ]:
# create beer profile df

In [ ]:
# create beer flavor profile df